# Convert ROOT files Clusterfeature

In [1]:
distances = [15]
doubleplanes = [12,30]
energies = [600]
erels = [500]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["inclxx"]
subruns = range(20)

In [2]:
import h5py
import numpy as np
import os
import sys
import joblib
import ROOT

sys.path.append("..")
from helpers import filename_for

ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


In [3]:
def extract(cluster, i_event, prim):
    return [
        i_event,
        prim,
        cluster.GetT(),
        cluster.GetE(),
        cluster.GetSize(),
        cluster.GetEToF(),
        cluster.GetEnergyMoment(),
        cluster.GetLastHit().GetT() - cluster.GetFirstHit().GetT(),
        cluster.GetMaxEnergyHit().GetE(),
        cluster.GetPosition().X(),
        cluster.GetPosition().Y(),
        cluster.GetPosition().Z(),
    ]

In [4]:
def get_entries(filename):
    # List Comprehension does not work with ROOT
    tfile = ROOT.TFile.Open(os.fspath(filename))
    ttree = tfile.Get("evt")
    num_events = ttree.GetEntries()
    num_clusters = 0
    for event in ttree:
        num_clusters += event.NeulandSecondaryClusters.GetEntries()
        num_clusters += event.NeulandPrimaryClusters.GetEntries()
    tfile.Close()
    return num_events, num_clusters

In [5]:
def root2hdf5(distance, doubleplane, energy, erel, neutron, physics, subrun):
    inpfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "digi.root")
    outfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "clusterfeature.h5")

    with h5py.File(outfile, "w") as h5file:
        num_events, num_clusters = get_entries(inpfile)

        chunk_size = 100
        num_clusterfeatures = 12
        clusters = h5file.create_dataset(
            "clusters",
            shape=(num_clusters, num_clusterfeatures),
            dtype=np.float32,
            chunks=(100, num_clusterfeatures),
            compression="gzip",
            compression_opts=9,
        )
        buff = np.zeros((num_clusterfeatures), np.float32)

        consolidated = h5file.create_dataset("consolidated", shape=(num_events, 3), dtype=np.int16)
        cbuff = np.zeros((3), np.int16)

        multiplicity = h5file.create_dataset("multiplicity", (num_events, 3), np.int8)
        mbuff = np.zeros((3), np.int8)

        i_event = 0
        i_cluster = 0
        tfile = ROOT.TFile.Open(os.fspath(inpfile))
        ttree = tfile.Get("evt")
        for event in ttree:
            # Consolidated Features
            # nHits: Number of hits
            cbuff[0] = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            cbuff[1] = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            cbuff[2] = round(sum([hit.GetE() for hit in event.NeulandHits]))
            consolidated[i_event] = cbuff

            # Multiplicity
            # nPN: Number of incoming primary neutrons
            mbuff[0] = event.NeulandPrimaryTracks.GetEntries()
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            mbuff[1] = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            mbuff[2] = event.NeulandPrimaryHits.GetEntries()
            multiplicity[i_event] = mbuff

            for cluster in event.NeulandSecondaryClusters:
                clusters[i_cluster] = extract(cluster, i_event, 0)
                i_cluster += 1

            for cluster in event.NeulandPrimaryClusters:
                clusters[i_cluster] = extract(cluster, i_event, 1)
                i_cluster += 1

            i_event += 1
        tfile.Close()

In [6]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing")(
    joblib.delayed(root2hdf5)(
        distance=distance,
        doubleplane=doubleplane,
        energy=energy,
        erel=erel,
        neutron=neutron,
        physics=physics,
        subrun=subrun,
    )
    for distance in distances
    for energy in energies
    for doubleplane in doubleplanes
    for neutron in neutrons
    for erel in erels
    for physics in physicss
    for subrun in subruns
)

print("Done")

Done
